# ddi - Mini-Challenge zu LE3, NoSQL
Roman Studer, Simon Luder

## Use Case

## Datenmodell

## Setup MongoDB

In [174]:
#!pip install pymongo
from pymongo import MongoClient
import pandas as pd
import pymongo


### Prepare data

In [175]:
def csv_to_dict(path):
    '''converts a csv file to a dictionary'''
    data = pd.read_csv(path)
    data["time_start"] = data["time_start"].str.replace('T', ' ')
    data["time_start"] = data["time_start"].str.replace('Z', '')
    data["time_start"] = data["time_start"].str.replace('.000', '')
    pd.to_datetime(data.time_start, format="%Y-%m-%d %H:%M:%S")
    data[["time_start", "arithmeticMean_10m"]].to_dict()

    dictionary = dict(zip(data["time_start"], data["arithmeticMean_10m"]))
    dictionary["sensorId"] = data["sensorId"][0]
    dictionary["lat"] = data["lat"][0]
    dictionary["lon"] = data["lon"][0]
    dictionary["boxName"] = data["boxName"][0]
    dictionary["boxId"] = data["boxId"][0]
    dictionary["unit"] = data["unit"][0]
    return dictionary

### Create db

In [176]:
mongo_client = pymongo.MongoClient('localhost', 27017)
mongo_db = mongo_client["ddi_mc2"]

### Define collections

In [177]:
air_pressure = mongo_db['air_pressure']
wind_force = mongo_db['wind_force']
wind_direction = mongo_db['wind_direction']
wind_speed = mongo_db['wind_speed']




In [178]:
path = './data/opensensemap_org-arithmeticMean-Windst%C3%A4rke-lat-lon-boxName-boxId-unit-20210518_115929.csv'
dictionary = csv_to_dict(path)
mongo_db.wind_force.insert_one(dictionary)

C:\Users\samlu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [179]:
mongo_client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [180]:
mongo_client.list_database_names()

['admin', 'config', 'ddi_mc2', 'local']

In [181]:
dblist = mongo_client.list_database_names()
if "ddi_mc2" in dblist:
    print("The database exists.")

The database exists.


In [184]:
print(mongo_db.wind_force.find)

<bound method Collection.find of Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ddi_mc2'), 'wind_force')>


## Setup InfluxDB 

In [3]:
#!pip install influxdb
from influxdb import InfluxDBClient
influx_client = InfluxDBClient(host='localhost', port=8086)